# 0. Set-Ups

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import DataFrame

In [ ]:
spark: SparkSession = SparkSession \
    .builder \
    .appName("Coalesce") \
    .master("local[4]") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 21:20:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> DataFrame:
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

# How coalesce works

In [4]:
num_rows = 2000000000
sdf1 = sdf_generator(num_rows, 4)

In [5]:
sdf1.rdd.getNumPartitions()

4

## Baseline

In [6]:
row_count1 = sdf1.count()
sdf_part1 = sdf1.withColumn("partition_id", f.spark_partition_id())
sdf_part_count1 = sdf_part1.groupBy("partition_id").count()
sdf_part_count1 = sdf_part_count1.withColumn("count_perc", 100*f.col("count")/row_count1)
sdf_part_count1.show()

+------------+---------+----------+
|partition_id|    count|count_perc|
+------------+---------+----------+
|           0|500000000|      25.0|
|           1|500000000|      25.0|
|           2|500000000|      25.0|
|           3|500000000|      25.0|
+------------+---------+----------+



In [7]:
sc.setJobDescription("Baseline 4 partitions")
sdf1.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

## Coalesce

In [8]:
sdf2 = sdf1.coalesce(3)
sdf2.rdd.getNumPartitions()

3

In [9]:
row_count2 = sdf2.count()
sdf_part2 = sdf2.withColumn("partition_id", f.spark_partition_id())
sdf_part_count2 = sdf_part2.groupBy("partition_id").count()
sdf_part_count2 = sdf_part_count2.withColumn("count_perc", 100*f.col("count")/row_count2)
sdf_part_count2.show()

+------------+----------+----------+
|partition_id|     count|count_perc|
+------------+----------+----------+
|           0| 500000000|      25.0|
|           1| 500000000|      25.0|
|           2|1000000000|      50.0|
+------------+----------+----------+



In [11]:
sdf2.rdd.getNumPartitions()

3

In [12]:
sdf2.coalesce(2).rdd.getNumPartitions()

2

In [13]:
sdf2.coalesce(1).rdd.getNumPartitions()

1

In [14]:
sdf2.coalesce(5).rdd.getNumPartitions()

4

In [10]:
sc.setJobDescription("Coalesce from 4 to 3")
sdf2.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

In [ ]:
sdf3 = sdf_generator(num_rows, 3)
sc.setJobDescription("3 Partitions")
sdf3.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

3


In [16]:
print(sdf3.rdd.getNumPartitions())

3


In [17]:
sdf4 = sdf_generator(num_rows, 8)
print(sdf4.rdd.getNumPartitions())
sc.setJobDescription("8 Partitions")
sdf4.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

8


In [18]:
sdf5 = sdf4.coalesce(4)
print(sdf5.rdd.getNumPartitions())
sc.setJobDescription("Coalesce from 8 to 4")
sdf5.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

4


In [19]:
sdf6 = sdf_generator(num_rows, 200001)
print(sdf6.rdd.getNumPartitions())
sc.setJobDescription("200001 partitions")
sdf6.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

200001


In [20]:
sdf7 = sdf6.coalesce(4)
print(sdf7.rdd.getNumPartitions())
sc.setJobDescription("Coalesce from 200001 to 4")
sdf7.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

4


24/12/09 22:45:25 WARN TaskSetManager: Stage 18 contains a task of very large size (2449 KiB). The maximum recommended task size is 1000 KiB.


In [21]:
sdf8 = sdf_generator(num_rows, 40)
print(sdf8.rdd.getNumPartitions())
sc.setJobDescription("40 partitions")
sdf8.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

40


# 3. Repartition

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import DataFrame

In [2]:
spark = SparkSession \
    .builder \
    .appName("Repartition") \
    .master("local[4]") \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 00:07:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def sdf_generator(num_rows: int, num_partitions: int = None) -> DataFrame:
    return (
        spark.range(num_rows, numPartitions=num_partitions)
        .withColumn("date", f.current_date())
        .withColumn("timestamp",f.current_timestamp())
        .withColumn("idstring", f.col("id").cast("string"))
        .withColumn("idfirst", f.col("idstring").substr(0,1))
        .withColumn("idlast", f.col("idstring").substr(-1,1))
        )

In [4]:
sdf_gen = sdf_generator(20)
sdf_gen.count()

20

In [5]:
sdf_gen.show()

+---+----------+--------------------+--------+-------+------+
| id|      date|           timestamp|idstring|idfirst|idlast|
+---+----------+--------------------+--------+-------+------+
|  0|2024-12-11|2024-12-11 00:08:...|       0|      0|     0|
|  1|2024-12-11|2024-12-11 00:08:...|       1|      1|     1|
|  2|2024-12-11|2024-12-11 00:08:...|       2|      2|     2|
|  3|2024-12-11|2024-12-11 00:08:...|       3|      3|     3|
|  4|2024-12-11|2024-12-11 00:08:...|       4|      4|     4|
|  5|2024-12-11|2024-12-11 00:08:...|       5|      5|     5|
|  6|2024-12-11|2024-12-11 00:08:...|       6|      6|     6|
|  7|2024-12-11|2024-12-11 00:08:...|       7|      7|     7|
|  8|2024-12-11|2024-12-11 00:08:...|       8|      8|     8|
|  9|2024-12-11|2024-12-11 00:08:...|       9|      9|     9|
| 10|2024-12-11|2024-12-11 00:08:...|      10|      1|     0|
| 11|2024-12-11|2024-12-11 00:08:...|      11|      1|     1|
| 12|2024-12-11|2024-12-11 00:08:...|      12|      1|     2|
| 13|202

In [6]:
def rows_per_partition(sdf: "DataFrame", num_rows: int) -> None:
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id").count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id").show()

rows_per_partition(sdf_gen, 20)

+------------+-----+----------+
|partition_id|count|count_perc|
+------------+-----+----------+
|           0|    5|      25.0|
|           1|    5|      25.0|
|           2|    5|      25.0|
|           3|    5|      25.0|
+------------+-----+----------+



In [7]:
def rows_per_partition_col(sdf: "DataFrame", num_rows: int, col: str) -> None:
    sdf_part = sdf.withColumn("partition_id", f.spark_partition_id())
    sdf_part_count = sdf_part.groupBy("partition_id", col).count()
    sdf_part_count = sdf_part_count.withColumn("count_perc", 100*f.col("count")/num_rows)
    sdf_part_count.orderBy("partition_id", col).show()

rows_per_partition_col(sdf_gen, 20, "idfirst")

+------------+-------+-----+----------+
|partition_id|idfirst|count|count_perc|
+------------+-------+-----+----------+
|           0|      0|    1|       5.0|
|           0|      1|    1|       5.0|
|           0|      2|    1|       5.0|
|           0|      3|    1|       5.0|
|           0|      4|    1|       5.0|
|           1|      5|    1|       5.0|
|           1|      6|    1|       5.0|
|           1|      7|    1|       5.0|
|           1|      8|    1|       5.0|
|           1|      9|    1|       5.0|
|           2|      1|    5|      25.0|
|           3|      1|    5|      25.0|
+------------+-------+-----+----------+



In [8]:
sdf1 = sdf_generator(num_rows=20000, num_partitions=4)
sdf1.rdd.getNumPartitions()

4

In [9]:
row_count = sdf1.count()
print(row_count)

20000


In [10]:
rows_per_partition(sdf1, num_rows)

NameError: name 'num_rows' is not defined

In [ ]:
rows_per_partition_col(sdf1, num_rows, "idfirst")

NameError: name 'num_rows' is not defined

24/12/11 03:25:29 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 933877 ms exceeds timeout 120000 ms
24/12/11 03:25:29 WARN SparkContext: Killing executors is not supported by current scheduler.
24/12/11 03:25:30 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [90]:
sc.setJobDescription("Baseline 4 partitions")
sdf1.write.format("noop").mode("overwrite").save()
sc.setJobDescription("None")

In [91]:
sdf_3 = sdf1.repartition(3)
sdf_3.rdd.getNumPartitions()

3

In [92]:
rows_per_partition(sdf_3, num_rows)

+------------+-----+----------+
|partition_id|count|count_perc|
+------------+-----+----------+
|           0| 6667| 3.3335E-4|
|           1| 6667| 3.3335E-4|
|           2| 6666|  3.333E-4|
+------------+-----+----------+



In [93]:
sdf_12 = sdf1.repartition(12)
sdf_12.rdd.getNumPartitions()

12

In [94]:
rows_per_partition(sdf_12, num_rows)

+------------+-----+----------+
|partition_id|count|count_perc|
+------------+-----+----------+
|           0| 1667|  8.335E-5|
|           1| 1666|   8.33E-5|
|           2| 1666|   8.33E-5|
|           3| 1666|   8.33E-5|
|           4| 1667|  8.335E-5|
|           5| 1667|  8.335E-5|
|           6| 1667|  8.335E-5|
|           7| 1667|  8.335E-5|
|           8| 1666|   8.33E-5|
|           9| 1667|  8.335E-5|
|          10| 1667|  8.335E-5|
|          11| 1667|  8.335E-5|
+------------+-----+----------+



In [107]:
spark.conf.set("spark.sql.shuffle.partitions", 200)
sdf_col_200 = sdf1.repartition(200, "idfirst")
sdf_col_200.rdd.getNumPartitions()

200

In [108]:
rows_per_partition(sdf_col_200, num_rows)

+------------+-----+----------+
|partition_id|count|count_perc|
+------------+-----+----------+
|           3| 1111|  5.555E-5|
|          18| 1111|  5.555E-5|
|          26| 1111|  5.555E-5|
|          35|    1|    5.0E-8|
|          49| 1111|  5.555E-5|
|          75| 1111|  5.555E-5|
|         139| 1111|  5.555E-5|
|         144|11111| 5.5555E-4|
|         166| 1111|  5.555E-5|
|         189| 1111|  5.555E-5|
+------------+-----+----------+



In [109]:
rows_per_partition_col(sdf_col_200, num_rows, "idfirst")


+------------+-------+-----+----------+
|partition_id|idfirst|count|count_perc|
+------------+-------+-----+----------+
|           3|      7| 1111|  5.555E-5|
|          18|      3| 1111|  5.555E-5|
|          26|      8| 1111|  5.555E-5|
|          35|      0|    1|    5.0E-8|
|          49|      5| 1111|  5.555E-5|
|          75|      6| 1111|  5.555E-5|
|         139|      9| 1111|  5.555E-5|
|         144|      1|11111| 5.5555E-4|
|         166|      4| 1111|  5.555E-5|
|         189|      2| 1111|  5.555E-5|
+------------+-------+-----+----------+



In [111]:
spark.conf.set("spark.sql.shuffle.partitions", 20)
sdf_col_20 = sdf1.repartition("idfirst")
sdf_col_20.rdd.getNumPartitions()

1